In [1]:
# импортируем необходимые библиотеки и модули
import pandas as pd
import numpy as np
import optuna
import random
from functools import partial
from pathlib import Path
from typing import Optional
import typer
import os

# импортируем из библиотеки etna нужные классы
from etna.datasets import TSDataset, generate_ar_df
from etna.loggers import WandbLogger,tslogger
from etna.metrics import MAE, MSE, SMAPE, Sign
from etna.models import CatBoostMultiSegmentModel
from etna.pipeline import Pipeline
from etna.transforms import (LagTransform,
                             SegmentEncoderTransform,
                             StandardScalerTransform)

# импортируем модуль предупреждений
import warnings
# включим игнорирование предупреждений
warnings.filterwarnings('ignore')

/Users/artemgruzdev/anaconda3/lib/python3.10/site-packages/etna/settings.py:61: UserWarning: `tsfresh` is not available, to install it, run `pip install tsfresh==0.19.0 && pip install protobuf==3.20.1`
  warnings.warn(


In [2]:
# функция установки начального значения генератора случайных чисел
def set_seed(seed: int = 42):
    # установка генератора случайных чисел python 
    random.seed(seed)
    # установка генератора случайных чисел numpy
    np.random.seed(seed)

In [3]:
def init_logger(config: dict, 
                project: str = 'wandb-sweeps', 
                tags: Optional[list] = ['test', 'sweeps']):
    # создание пустого списка для хранения обьектов WandbLogge
    tslogger.loggers = []
    # создание экземпляра WandbLogge. W&B предоставляет облегченную оболочку для 
    # регистрации экспериментов машинного обучения
    wblogger = WandbLogger(project=project, tags=tags, config=config)
    # добавление обьекта WandbLogge в список
    tslogger.add(wblogger)

In [4]:
def dataloader(file_path: Path, freq: str = 'D') -> TSDataset:
    df = pd.read_csv(file_path)
    df = TSDataset.to_dataset(df)
    ts = TSDataset(df=df, freq=freq)
    return ts

In [5]:
# целевая функция Optuna
def objective(trial: optuna.Trial, metric_name: str, ts: TSDataset, horizon: int, lags: int, seed: int):

    # установка начального значения генератора случайных чисел 
    set_seed(seed)

    #  создадим экземпляр Pipeline. Определить модель и характеристики
    pipeline = Pipeline(
        # определим модель
        model=CatBoostMultiSegmentModel(
            iterations=trial.suggest_int('iterations', 10, 100),
            depth=trial.suggest_int('depth', 1, 12),
        ),
        # определим список трансформеров
        transforms=[
            StandardScalerTransform('target'),
            SegmentEncoderTransform(),
            LagTransform(in_column='target', 
                         lags=list(range(horizon, horizon + 
                                         trial.suggest_int('lags', 1, lags)))),
        ],
        # установка горизонта для прогнозов
        horizon=horizon,
    )

    # инициализируем WandB logger
    init_logger(pipeline.to_dict())

    # запускаем перекрестную проверку с подсчетом метрик
    metrics, _, _ = pipeline.backtest(ts=ts, metrics=[MAE(), SMAPE(), Sign(), MSE()])
    
    # возвращаем средние значения по найденным метрикам
    return metrics[metric_name].mean()


In [6]:
# функция запуска оптимизации Optuna
def run_optuna(
    # горизонт для прогнозов
    horizon: int = 14,
    # метрика
    metric_name: str = 'MAE',
    # URL-адрес базы данных,
    storage: str = 'sqlite:///optuna.db',
    # название 
    study_name: Optional[str] = None,
    # количество проходов
    n_trials: int = 5,
    # путь к файлу данных
    file_path: Path = 'Data/example_dataset.csv',
    # направление оптимизации
    direction: str = 'minimize',
    # частота временного ряда
    freq: str = 'D',
    #  количество лагов
    lags: int = 24,
    # стартовое значение генератора псевдослучайных чисел
    seed: int = 11,
):

    # загрузка данных
    ts = dataloader(file_path, freq=freq)

    # создание сессии оптимизации Optuna
    study = optuna.create_study(
        # URL базы данных
        storage=storage,
        study_name=study_name,
        # семплер
        sampler=optuna.samplers.TPESampler(multivariate=True, group=True),
        # флаг управления поведением при разрешении конфликта
        load_if_exists=True,
        # направление оптимизации
        direction=direction,
    )

    # запуск оптимизации Optuna
    study.optimize(
        partial(objective, metric_name=metric_name, ts=ts, horizon=horizon, 
                lags=lags, seed=seed), n_trials=n_trials
        )


In [7]:
# Вызов функции run_optuna, запуск оптимизации
run_optuna(file_path='Data/example_dataset.csv')

[I 2023-05-24 15:07:16,322] A new study created in RDB with name: no-name-1c0d6852-0d84-4270-9ee2-b80e2a0a24ec
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /Users/artemgruzdev/.netrc


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   55.5s remaining:    0.0s


wandb: Currently logged in as: gewissta. Use `wandb login --relogin` to force relogin


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.1min remaining:    0.0s


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.6min finished


MAE_mean,▁
MAE_median,▁
MAE_percentile_25,▁
MAE_percentile_5,▁
MAE_percentile_75,▁
MAE_percentile_95,▁
MAE_std,▁
MSE_mean,▁
MSE_median,▁
MSE_percentile_25,▁
MSE_percentile_5,▁


[I 2023-05-24 15:40:40,102] Trial 4 finished with value: 47.90122447796802 and parameters: {'iterations': 68, 'depth': 2, 'lags': 23}. Best is trial 1 with value: 43.30188972892675.
